In [45]:
!rm -rf spark-3.1.1-bin-hadoop3.2

In [46]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q findspark pyspark

In [47]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [48]:
!ls

sample_data


In [49]:
!pip show pyspark

Name: pyspark
Version: 3.5.1
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: py4j
Required-by: 


In [50]:
!pip install graphframes

In [51]:
!python -V

Python 3.10.12


In [52]:

!curl -L -o "/usr/local/lib/python3.10/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.3.2-s_2.11.jar" https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.1-s_2.12/graphframes-0.8.2-spark3.1-s_2.12.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  242k  100  242k    0     0   713k      0 --:--:-- --:--:-- --:--:--  714k


In [53]:
!ls /usr/local/lib/python3.10/dist-packages/pyspark/jars/

activation-1.1.1.jar
aircompressor-0.26.jar
algebra_2.12-2.0.1.jar
annotations-17.0.0.jar
antlr4-runtime-4.9.3.jar
antlr-runtime-3.5.2.jar
aopalliance-repackaged-2.6.1.jar
arpack-3.0.3.jar
arpack_combined_all-0.1.jar
arrow-format-12.0.1.jar
arrow-memory-core-12.0.1.jar
arrow-memory-netty-12.0.1.jar
arrow-vector-12.0.1.jar
audience-annotations-0.5.0.jar
avro-1.11.2.jar
avro-ipc-1.11.2.jar
avro-mapred-1.11.2.jar
blas-3.0.3.jar
bonecp-0.8.0.RELEASE.jar
breeze_2.12-2.1.0.jar
breeze-macros_2.12-2.1.0.jar
cats-kernel_2.12-2.1.1.jar
chill_2.12-0.10.0.jar
chill-java-0.10.0.jar
commons-cli-1.5.0.jar
commons-codec-1.16.0.jar
commons-collections-3.2.2.jar
commons-collections4-4.4.jar
commons-compiler-3.1.9.jar
commons-compress-1.23.0.jar
commons-crypto-1.1.0.jar
commons-dbcp-1.4.jar
commons-io-2.13.0.jar
commons-lang-2.6.jar
commons-lang3-3.12.0.jar
commons-logging-1.1.3.jar
commons-math3-3.6.1.jar
commons-pool-1.5.4.jar
commons-text-1.10.0.jar
compress-lzf-1.1.2.jar
curator-client-2.13.0.jar
cur

In [54]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.jars", "/usr/local/lib/python3.10/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.3.2-s_2.11.jar") \
    .getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)  # Property used to format output tables better\


In [55]:
from graphframes import *
from graphframes import GraphFrame

In [56]:
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

PySpark Version :3.5.1
PySpark Version :3.5.1


In [57]:
input_path = "https://github.com/gauravsharma2/BIG-DATA-GRAPHX"
output_path = "/content/sample_data/

In [ ]:
spark = SparkSession.builder.appName("BIG DATA ASSIGNMENT_3").getOrCreate()
edges = spark.read.option("header","true").option("inferedSchema","true").option("delimiter","\t").csv(input_path)
edges = edges.select(col("FromNodeId").alias("src"), col("ToNodeId").alias("dst"))
display(edges)

In [ ]:
vertices = edges.select(col("src").alias("id")).union(edges.select(col("dst").alias("id"))).distinct()
display(vertices)

In [ ]:
G = GraphFrame(vertices,edges)

In [82]:
# QUERY : a. Find the top 5 nodes with the highest outdegree and find the count of the number of outgoing edges in each.
# ----------------------------------------------------------------------------------------------------------------------
od = G.outDegrees.orderBy(desc("outDegree")).take(5)
odf = spark.createDataFrame(od)
odf.write.mode("overwrite").option("header", "true").csv(o + "Query Outputs/OPTION A")
display(od)

[Row(id='2565', outDegree=893),
 Row(id='766', outDegree=773),
 Row(id='11', outDegree=743),
 Row(id='457', outDegree=732),
 Row(id='2688', outDegree=618)]

In [84]:
# QUERY : b. Find the top 5 nodes with the highest indegree and find the count of the number of incoming edges in each.
# ---------------------------------------------------------------------------------------------------------------------
ind = G.inDegrees.orderBy(desc("inDegree")).take(5)
indd = spark.createDataFrame(ind)
indd.write.mode("overwrite").option("header", "true").csv(o + "Query Outputs/OPTION B")
display(ind)

[Row(id='4037', inDegree=457),
 Row(id='15', inDegree=361),
 Row(id='2398', inDegree=340),
 Row(id='2625', inDegree=331),
 Row(id='1297', inDegree=309)]

In [89]:
# QUERY : c. Calculate PageRank for each of the nodes and output the top 5 nodes with the highest PageRank values.
# ----------------------------------------------------------------------------------------------------------------
pr = G.pageRank(resetProbability=0.15, maxIter=10)
pro = pr.vertices.orderBy(desc("pagerank")).select("id", "pagerank").distinct()
pr5 = pro.orderBy(desc("pagerank")).take(5)
prd = spark.createDataFrame(pr5)
prd.write.mode("overwrite").option("header", "true").csv(o + "Query Outputs/OPTION C")
display(pr5)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")
/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


[Row(id='4037', pagerank=32.7613925903508),
 Row(id='15', pagerank=26.253004957619474),
 Row(id='6634', pagerank=26.164524434886495),
 Row(id='2625', pagerank=23.511515933026384),
 Row(id='2398', pagerank=18.728389390669687)]

In [87]:
# QUERY : d. Run the connected components algorithm on it and find the top 5 components with the largest number of nodes.
# -----------------------------------------------------------------------------------------------------------------------
spark.sparkContext.setCheckpointDir("/tmp/checkpoints")
sp = GraphFrame(vertices, edges.sample(False, 0.1))
cc = sp.connectedComponents()
cc5 = cc.groupBy("component").count().orderBy(desc("count")).take(5)
ccd = spark.createDataFrame(cc5)
ccd.write.mode("overwrite").option("header", "true").csv(output_path + "Query Outputs/OPTION D")
display(cc5)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


[Row(component=1, count=3500),
 Row(component=893353197600, count=3),
 Row(component=1125281431566, count=3),
 Row(component=721554505733, count=2),
 Row(component=987842478110, count=2)]

In [88]:
# QUERY : e. Run the triangle counts algorithm on each of the vertices and output the top 5 vertices with the largest triangle count.
# -----------------------------------------------------------------------------------------------------------------------------------
tc = plot.triangleCount()
tc5 = tc.select("id", "count").distinct().orderBy(desc("count")).take(5)
tcd = spark.createDataFrame(tc5)
tcd.write.mode("overwrite").option("header", "true").csv(output_path + "Query Outputs/OPTION E")
display(tc5)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


[Row(id='2565', count=30940),
 Row(id='1549', count=22003),
 Row(id='766', count=18204),
 Row(id='1166', count=17361),
 Row(id='2688', count=14220)]